In [8]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean, stddev, min, max, last, count, countDistinct


In [2]:
# Some global constants
# NAN_VALUE = -127
CAT_VARS = ["B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_66","D_68"]
TARGET_COLUMN = 'target'
TRAIN_FILE = '../Data/train_small.csv'
TEST_FILE = '../Data/test_small.csv'

In [3]:
sc = SparkContext(appName="Train and Save Models")
sc.setLogLevel("ERROR")
spark = SparkSession(sc)
data_train = spark.read.csv(TRAIN_FILE, header=True, inferSchema=True)
data_test = spark.read.csv(TEST_FILE, header=True, inferSchema=True)
data_train = data_train.fillna(-1)
data_test = data_test.fillna(-1)

your 131072x1 screen size is bogus. expect trouble
24/04/11 08:19:24 WARN Utils: Your hostname, DESKTOP-D1SJQQ6 resolves to a loopback address: 127.0.1.1; using 172.27.228.206 instead (on interface eth0)
24/04/11 08:19:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/11 08:19:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
def feature_engineer_spark(df, CAT_VARS):
    all_cols = [c for c in df.columns if c not in ['customer_ID', 'S_2']]
    print('shape before engineering:', (df.count(), len(df.columns)))
    print('columns:', all_cols)
    cont_vars = [c for c in all_cols if c not in CAT_VARS]
    print('continuous variables:', cont_vars)
    
    cont_vars_agg_exprs = [expr for c in cont_vars for expr in (
        mean(c).alias(c + '_mean'),
        stddev(c).alias(c + '_std'),
        min(c).alias(c + '_min'),
        max(c).alias(c + '_max'),
        last(c).alias(c + '_last')
    )]
    print('cont_vars_agg_exprs:', cont_vars_agg_exprs)
    cont_vars_agg = df.groupBy("customer_ID").agg(*cont_vars_agg_exprs)
    print('cont_vars_agg:', cont_vars_agg)

    cat_vars_agg_exprs = [expr for c in CAT_VARS for expr in (
        count(c).alias(c + '_count'),
        last(c).alias(c + '_last'),
        countDistinct(c).alias(c + '_nunique')
    )]
    print('cat_vars_agg_exprs:', cat_vars_agg_exprs)
    cat_vars_agg = df.groupBy("customer_ID").agg(*cat_vars_agg_exprs)
    df_agg = cont_vars_agg.join(cat_vars_agg, "customer_ID")
    print('shape after engineering:', (df_agg.count(), len(df_agg.columns)))
    return df_agg

data_train = feature_engineer_spark(data_train, CAT_VARS)
data_test = feature_engineer_spark(data_test, CAT_VARS)

shape before engineering: (43959, 191)
columns: ['P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41', 'B_3', 'D_42', 'D_43', 'D_44', 'B_4', 'D_45', 'B_5', 'R_2', 'D_46', 'D_47', 'D_48', 'D_49', 'B_6', 'B_7', 'B_8', 'D_50', 'D_51', 'B_9', 'R_3', 'D_52', 'P_3', 'B_10', 'D_53', 'S_5', 'B_11', 'S_6', 'D_54', 'R_4', 'S_7', 'B_12', 'S_8', 'D_55', 'D_56', 'B_13', 'R_5', 'D_58', 'S_9', 'B_14', 'D_59', 'D_60', 'D_61', 'B_15', 'S_11', 'D_62', 'D_63', 'D_64', 'D_65', 'B_16', 'B_17', 'B_18', 'B_19', 'D_66', 'B_20', 'D_68', 'S_12', 'R_6', 'S_13', 'B_21', 'D_69', 'B_22', 'D_70', 'D_71', 'D_72', 'S_15', 'B_23', 'D_73', 'P_4', 'D_74', 'D_75', 'D_76', 'B_24', 'R_7', 'D_77', 'B_25', 'B_26', 'D_78', 'D_79', 'R_8', 'R_9', 'S_16', 'D_80', 'R_10', 'R_11', 'B_27', 'D_81', 'D_82', 'S_17', 'R_12', 'B_28', 'R_13', 'D_83', 'R_14', 'R_15', 'D_84', 'R_16', 'B_29', 'B_30', 'S_18', 'D_86', 'D_87', 'R_17', 'R_18', 'D_88', 'B_31', 'S_19', 'R_19', 'B_32', 'S_20', 'R_20', 'R_21', 'B_33', 'D_89', 'R_22', 'R_23', 'D_91', 'D

In [11]:

filtered_data_train = data_train.filter(data_train.P_2_std.isNotNull())
filtered_data_train.head(5)

24/04/11 08:37:52 WARN DAGScheduler: Broadcasting large task binary with size 1954.4 KiB


[Row(customer_ID='00d769dbf0a5e3e73a8223566b33adc5686954dd337359e6f18b7991f116eed4', P_2_mean=1.0031298655148135, P_2_std=0.005436013085450345, P_2_min=0.9992860237994728, P_2_max=1.0069737072301543, P_2_last=0.9992860237994728, D_39_mean=0.1523952541721795, D_39_std=0.07688082026338816, D_39_min=0.0980323048207536, D_39_max=0.2067582035236054, D_39_last=0.2067582035236054, B_1_mean=0.06094031238715415, B_1_std=0.007528486958188751, B_1_min=0.0556168682069444, B_1_max=0.0662637565673639, B_1_last=0.0662637565673639, B_2_mean=0.3831369369440982, B_2_std=0.025222422069795934, B_2_min=0.3653019912605963, B_2_max=0.4009718826276002, B_2_last=0.3653019912605963, R_1_mean=0.0039927102841037995, R_1_std=0.0002911748781010233, R_1_min=0.0037868185532874, R_1_max=0.0041986020149202, R_1_last=0.0041986020149202, S_3_mean=0.1058009394889505, S_3_std=0.004979489413113101, S_3_min=0.1022799087580916, S_3_max=0.1093219702198094, S_3_last=0.1022799087580916, D_41_mean=0.00784872317323485, D_41_std=0.